In [ ]:
# 创建 TensorBoard 回调
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import LearningRateScheduler, TensorBoard, EarlyStopping
#学习率衰减
reduce_lr = ReduceLROnPlateau(monitor='val_loss', patience=5, mode='auto',min_lr=0,factor=0.1)
#tensorboard日志
tensorboard = TensorBoard(log_dir='./log/logs_1d_cnn_res50', histogram_freq=0, write_graph=True, write_images=False)
###tensorboard --logdir=./log/logs_1d_cnn
#早停
early_stopping = EarlyStopping(monitor='val_loss',  
                                mode='min',          
                                patience=5,          
                                verbose=1)

# 训练模型
model.fit(X_train_3d, y_train, epochs=30, batch_size=16, validation_data=(X_test_3d, y_test),callbacks=[tensorboard,reduce_lr])

In [ ]:
import keras
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

# k折交叉验证
# num_folds=10
num_folds = 10
kf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)
X=data_np
Y=encoded_labels_df

y=indexed_labels

# 存储每次验证的结果
cv_scores = []


for fold, (train_index, val_index) in enumerate(kf.split(X, y)):
    print(f"第 {fold + 1} 折/{num_folds}")

    # 划分训练集和验证集
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]
  
    # 调整数据维度
    X_train_3 = np.expand_dims(X_train, axis=2)

    X_val_3 = np.expand_dims(X_val, axis=2)
    model=build_model()



    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # 训练模型
    model.fit(X_train_3, y_train, epochs=12, batch_size=16, validation_data=(X_val_3, y_val), verbose=1)

    # 在验证集上评估模型
    scores = model.evaluate(X_val_3, y_val, verbose=0)
    cv_scores.append(scores[1])
print(cv_scores)


In [ ]:
#使用GridSearchCV自动执行超参数优化

from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from keras.optimizers import Adam, RMSprop, SGD, Nadam

# 创建一个更复杂的 Keras 模型
def create_model(optimizer='adam', learning_rate=0.001, dropout_rate=0.0, neurons=16):
    # 根据选择的优化器调整学习率
    if optimizer == 'adam':
        optimizer = Adam(learning_rate=learning_rate)
    elif optimizer == 'rmsprop':
        optimizer = RMSprop(learning_rate=learning_rate)
    elif optimizer == 'sgd':
        optimizer = SGD(learning_rate=learning_rate)
    elif optimizer == 'nadam':
        optimizer = Nadam(learning_rate=learning_rate)

    model = Sequential()
    model.add(Conv1D(neurons, 3, activation='relu', input_shape=(bin_num, CNNch)))
    model.add(MaxPooling1D(2))

    model.add(Conv1D(neurons, 3, activation='relu'))
    model.add(MaxPooling1D(2))

    model.add(Conv1D(neurons, 3, activation='relu'))
    model.add(Conv1D(neurons, 3, activation='relu'))
    model.add(MaxPooling1D(2))

    model.add(Conv1D(neurons*4, 3, activation='relu'))
    model.add(Conv1D(neurons*4, 3, activation='relu'))
    model.add(MaxPooling1D(2))

    model.add(Conv1D(neurons*4, 3, activation='relu'))
    model.add(Conv1D(neurons*4, 3, activation='relu'))
    model.add(MaxPooling1D(2))

    model.add(Flatten())
    model.add(Dense(neurons*4, activation='relu'))
     # 添加 Dropout 层
    model.add(Dropout(dropout_rate))
    model.add(Dense(neurons*2, activation='relu'))
     # 添加 Dropout 层
    model.add(Dropout(dropout_rate))
    model.add(Dense(class_num, activation='softmax'))

    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model

# 转换模型为 KerasClassifier
model = KerasClassifier(build_fn=create_model, epochs=20, batch_size=16, verbose=0)

# 定义要搜索的超参数和其可能的取值组合
param_grid = {
    'optimizer': ['adam'],
    'learning_rate': [0.0001,0.001,0.01],
    'dropout_rate': [0.0, 0.2, 0.5],
    'neurons': [2,4, 8,16,32],
    'epochs': [30, 50,70],
    'batch_size': [8,16,32]
}

# 创建 GridSearchCV 实例
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_test_3d, y_test)

# 输出结果
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
